# Data Encoding, Decoding and Flow

## Apache Thrift

The thrift type system includes base types like _bool, byte, double, string and integer_ but also special types like _binary_ and _struct_ (like classes) and also containers (_list, set, map_) that correspond to commonly available interfaces in most programming languages.

Base types:

- bool: A boolean value (true or false)
- byte: An 8-bit signed integer
- i16: A 16-bit signed integer
- i32: A 32-bit signed integer
- i64: A 64-bit signed integer
- double: A 64-bit floating point number
- string: A text string encoded using UTF-8 encoding

Thrift type definitions are defined in `.thrift` files. The Thrift compiler generates code in various languages from the `.thrift` files.

### Encoding

Let's use the following example record (JSON or dictionary-like) to encode:

```json
{
  "userName": "Martin",
  "favoriteNumber": 1337,
  "interests": ["daydreaming", "hacking"]
}
```

We can encode the record in Thrift using the following schema in the `.thrift` file:

```thrift
struct Person {
  1: required string userName,
  2: optional i64 favoriteNumber,
  3: optional list<string> interests
}
```

Thrift comes with a code generation tool that takes a schema definition like the ones shown here, and produces classes that implement the schema in various programming languages. Our code can call this generated code to encode or decode records of the schema.

The data encoded with this schema looks like this:
![thrift_binary_protocol](../assets/thrift_binary_protocol.png)

Each field has a type annotation (to indicate whether it is a string, integer, list, etc.) and, where required, a length indication (length of a string, number of items in a list). The strings that appear in the data (“Martin”, “daydreaming”, “hacking”) are encoded as UTF-8.

There are no field names (userName, favoriteNumber, interests). Instead, the encoded data contains _field tags_, which are numbers (1, 2, and 3). Those are the numbers that appear in the schema definition. Field tags are like aliases for fields—they are a compact way of saying what field we’re talking about, without having to spell out the field name.

Next, let's add a service. A service is a collection of method interfaces that can be called remotely. A service is defined in a `.thrift` file like this:

```thrift
service School {
    Person teachCourse(1: required Person person, 2: required string course)
}
```

The first line declares a service called `School`. The second line declares a method called `teachCourses`, which takes two arguments: a `Person` record and a `string`. The method returns a `Person` record.

### RPC

Nows, let's look at how to use the generated code to make remote procedure calls. We will write codes for 2 sides of the server-client application- the client initiates an RPC call and waits for a response from the server. The server executes the requested operation and returns a response to the client.

Here, we use `%%writefile` magic command to write the code to a file instead of running it in the cell.

In [1]:
%%writefile ../schema/person.thrift

struct Person {
  1: required string userName,
  2: optional i64 favoriteNumber,
  3: optional list<string> interests
}

service School {
    Person teachCourse(1: required Person person, 2: required string course)
}

Overwriting ../schema/person.thrift


In [2]:
%%writefile ../person_thrift_server.py
import thriftpy2
person_thrift = thriftpy2.load("./schema/person.thrift", module_name="person_thrift")

from thriftpy2.rpc import make_server

class School(object):
    def teachCourse(self, person, course):
        person.interests.append(course)
        return person

server = make_server(person_thrift.School, School(), client_timeout=None)
server.serve()

Overwriting ../person_thrift_server.py


Then, run `python person_thrift_server.py` in a new terminal. This will start the server.

In [3]:
import thriftpy2
person_thrift = thriftpy2.load("../schema/person.thrift", module_name="person_thrift")

from thriftpy2.rpc import make_client

school = make_client(person_thrift.School, timeout=None)

In [4]:
martin = person_thrift.Person(
    userName="Martin", favoriteNumber=1337, interests=["daydreaming", "hacking"]
)

In [5]:
martin.interests

['daydreaming', 'hacking']

In [6]:
martin = school.teachCourse(martin, "coding")

In [7]:
martin.interests

['daydreaming', 'hacking', 'coding']

> 1. Add a new field `grade` (0-100) with an appropriate type annotation to the `Person` struct. Then, add a new method `assignGrade` to the `School` service that takes a `Person` record and a `grade` arguments, assigns the `grade` to the `Person` and returns the `Person`. Then call the method by passing `martin` and a grade number, and print his grade.
>
> 2. Add a method `teachCourses` to School to add a list of courses instead of just one course. Then pass `martin` and a list of course-- `["cooking", "sewing"]` to the method, and print his new interests.

## Protocol Buffers (Protobuf)

Protobuf types include:

- double: double precision floating point number
- float: single precision floating point number
- int32: signed integer, uses variable-length encoding
- int64: signed integer, uses variable-length encoding
- uint32: unsigned integer, uses variable-length encoding
- uint64: unsigned integer, uses variable-length encoding
- sint32: signed integer, uses variable-length encoding, more efficient than int32
- sint64: signed integer, uses variable-length encoding, more efficient than int64
- fixed32: unsigned integer, always 4 bytes
- fixed64: unsigned integer, always 8 bytes
- sfixed32: signed integer, always 4 bytes
- sfixed64: signed integer, always 8 bytes
- bool: boolean value
- string: UTF-8 text string
- bytes: sequence of bytes
- enum: enumerated type
- message: nested message type
- map: map type
- Any: dynamic type

Protobuf schema definitions are defined in `.proto` files. The Protobuf compiler generates code in various languages from the `.proto` files.

### Encoding

We can encode the previous example record in Protobuf using the following schema in the `.proto` file:

```protobuf
message Person {
  required string user_name = 1;
  optional int64 favorite_number = 2;
  repeated string interests = 3;
}
```

The data encoded with this schema looks like this:
![protobuf](../assets/protobuf.png)

Protocol Buffers have an interesting aspect regarding its datatype handling. Unlike having a specific list or array datatype, it utilizes a `repeated` marker for fields, which serves as a third option alongside `required` and `optional`.

As depicted in the figure, a repeated field is simply represented by the same field tag appearing multiple times in the record. The advantage of this approach is that converting an optional (single-valued) field into a repeated (multi-valued) field is permissible. When new code reads old data, it interprets it as a list with either zero or one element, depending on whether the field was present. On the other hand, old code reading new data only perceives the last element of the list.

### gRPC

Nows, let's look at how to use the generated code to make remote procedure calls. We will use gRPC, which is a high-performance RPC framework built on top of Protobuf. gRPC is a client-server application where the client initiates an RPC call and waits for a response from the server. The server executes the requested operation and returns a response to the client.

In [8]:
%%writefile ../schema/person.proto
syntax = "proto3";

message Person {
  string user_name = 1;
  optional int64 favorite_number = 2;
  repeated string interests = 3;
}

message CourseRequest {
  Person person = 1;
  string course = 2;
}


service School {
  rpc teachCourse(CourseRequest) returns (Person) {}
}

Overwriting ../schema/person.proto


Then, run the following command in a terminal to generate the Python code:

```bash
python -m grpc_tools.protoc -I./schema --python_out=. --grpc_python_out=. ./schema/person.proto
```

This will generate the following files:

```bash
person_pb2.py
person_pb2_grpc.py
```

In [9]:
%%writefile ../person_protobuf_server.py
from concurrent import futures
import grpc
import person_pb2_grpc


class School(person_pb2_grpc.SchoolServicer):

  def teachCourse(self, request, context):
    request.person.interests.append(request.course)
    return request.person

server = grpc.server(futures.ThreadPoolExecutor(max_workers=2))
person_pb2_grpc.add_SchoolServicer_to_server(
    School(), server)
server.add_insecure_port('[::]:50051')
server.start()
server.wait_for_termination()

Overwriting ../person_protobuf_server.py


Then, run `python person_protobuf_server.py` in a new terminal. This will start the server.

In [10]:
import sys
sys.path.append('..')
import grpc
import person_pb2
import person_pb2_grpc


In [11]:
def teach_course(stub, person, course):
    person = stub.teachCourse(person_pb2.CourseRequest(person=person, course=course))
    return person


with grpc.insecure_channel('localhost:50051') as channel:
    martin = person_pb2.Person(user_name='Martin', favorite_number=1337, interests=["daydreaming", "hacking"])
    course = "coding"
    stub = person_pb2_grpc.SchoolStub(channel)
    martin = teach_course(stub, martin, course)
    print(martin.interests)

['daydreaming', 'hacking', 'coding']


> Add a new field `grade` (0-100) with an appropriate type annotation to the `Person` message. Then, add a new method `assignGrade` to the `School` service that takes a `GradeRequest` message (which consists of a `Person` record and a `grade` arguments), assigns the `grade` to the `Person` and returns the `Person`. Then call the method by passing "Martin" `person` and a grade number, and print his grade.

## Schema Evolution for Thrift and Protobuf

Let's explore how Thrift and Protocol Buffers manage schema changes to ensure both _backward_ and _forward compatibility_.

From the examples above, an encoded record comprises concatenated encoded fields. Each field is identified by a tag number (e.g., 1, 2, 3 in the sample schemas) and annotated with a datatype (such as string or integer). If a field value is not set, it is simply omitted from the encoded record. It's important to note that field tags play a crucial role in determining the meaning of the encoded data. Although you can modify a field's name in the schema without issues, changing a field's tag would render all existing encoded data invalid.

Adding new fields to the schema is permissible, but it requires assigning each field a new tag number. Old code, which is unaware of the new tag numbers, can still read data written by new code, and if it encounters an unrecognized tag number, it can safely ignore that field. The datatype annotation allows the parser to determine the necessary bytes to skip, thus maintaining _forward compatibility_.

Regarding _backward compatibility_, as long as each field retains a unique tag number, new code can read old data successfully because the tag numbers retain their original meaning. However, when adding a new field, it cannot be made required since old code wouldn't have written this new field, causing compatibility issues. Hence, to ensure backward compatibility, **any fields added after the initial schema deployment must be optional or have default values**.

Removing a field follows a similar principle to adding one, but with backward and forward compatibility concerns reversed. Only optional fields can be removed, never required fields, and once a tag number is removed, it can't be reused to prevent conflicts with existing data written with the old tag number.

Changing the datatype of a field is possible, but it carries the risk of losing precision or truncating values. For instance, converting a 32-bit integer into a 64-bit integer may lead to data loss when old code reads data written by new code, as the old code uses a 32-bit variable to hold the value. If the 64-bit value exceeds the capacity of a 32-bit variable, it will be truncated. Thus, careful consideration is necessary when altering field datatypes, and consulting the documentation is advisable to understand potential implications.

## Apache Avro

Avro has the following types:

- null: no value
- boolean: a binary value
- int: 32-bit signed integer
- long: 64-bit signed integer
- float: single precision (32-bit) IEEE 754 floating-point number
- double: double precision (64-bit) IEEE 754 floating-point number
- bytes: sequence of 8-bit unsigned bytes
- string: Unicode character sequence
- record: ordered collection of named fields
- enum: enumeration of string values
- array: ordered collection of values
- map: collection of key-value pairs
- union: ordered list of values

It has two schema languages: one (`Avro IDL`) intended for human editing, and one (based on JSON) that is more easily machine-readable.

### Encoding

We can encode the previous example record in IDL using the following schema in the `.avsc` file:

```avro
record Person {
  string userName;
  union { null, long } favoriteNumber = null;
  array<string> interests;
}
```

The equivalent JSON representation of that schema is as follows:

```json
{
  "type": "record",
  "name": "Person",
  "fields": [
    { "name": "userName", "type": "string" },
    { "name": "favoriteNumber", "type": ["null", "long"], "default": null },
    { "name": "interests", "type": { "type": "array", "items": "string" } }
  ]
}
```

The data encoded with this schema looks like this:
![avro](../assets/avro.png)

First and foremost, it's important to note that the schema lacks tag numbers. When we encode our sample record using this schema, the resulting Avro binary encoding is impressively compact, spanning just _32 bytes_—the most space-efficient among all the encodings we've observed.

Examining the byte sequence, one can readily discern the _absence of field identifiers or datatype markers_. The encoding solely comprises concatenated values. For instance, a string is represented by a length prefix followed by UTF-8 bytes, but there are no explicit indicators within the encoded data to specify that it is, indeed, a string. In fact, it could be interpreted as an integer or any other data type altogether. Similarly, an integer is encoded using a variable-length encoding.

To correctly parse the binary data, you must traverse the fields in the order they appear in the schema and _refer to the schema_ itself to ascertain the datatype of each field. Consequently, the binary data can only be accurately decoded if the code reading the data employs the exact same schema as the code that wrote the data. Any deviation or mismatch in the schema between the reader and the writer would result in incorrectly decoded data.

With Avro, data encoding and decoding are based on two schemas: the `writer's schema` used during data encoding and the `reader's schema` employed during data decoding. These schemas do not necessarily have to be identical but should be compatible. When decoding data, the Avro library compares the writer's and reader's schemas, resolving any discrepancies between them.

The Avro specification ensures that fields in different orders between the writer's and reader's schemas pose no issues during resolution since schema matching occurs based on field names. If the reader's schema lacks a field present in the writer's schema, it is simply ignored. Conversely, if the reader's schema expects a field that the writer's schema does not contain, the missing field is filled in with a default value declared in the reader's schema. This allows for flexible schema evolution while maintaining data compatibility.

### Reading (Decoding) a File

Instead of demonstrating RPC, let's look at how to decode data from a file from a real-world dataset. We have a genomic variation data of 1000 samples from the [OpenCGA](http://docs.opencb.org/display/opencga/Welcome+to+OpenCGA) project.

In [12]:
import fastavro
import copy
import json
from pprint import pprint

In [13]:
with open('../data/1k.variants.avro', 'rb') as f:
    reader = fastavro.reader(f)
    genomic_var_1k = [sample for sample in reader]
    metadata = copy.deepcopy(reader.metadata)
    writer_schema = copy.deepcopy(reader.writer_schema)
    schema_from_file = json.loads(metadata['avro.schema'])

In [14]:
len(genomic_var_1k)

1000

In [15]:
pprint(writer_schema)

{'fields': [{'doc': '* The variant ID.',
             'name': 'id',
             'type': ['null',
                      {'avro.java.string': 'String', 'type': 'string'}]},
            {'default': [],
             'doc': '* Other names used for this genomic variation.',
             'name': 'names',
             'type': {'items': {'avro.java.string': 'String', 'type': 'string'},
                      'type': 'array'}},
            {'doc': '* Chromosome where the genomic variation occurred.',
             'name': 'chromosome',
             'type': {'avro.java.string': 'String', 'type': 'string'}},
            {'doc': '* Normalized position where the genomic variation '
                    'starts.\n'
                    '         * <ul>\n'
                    '         * <li>SNVs have the same start and end '
                    'position</li>\n'
                    '         * <li>Insertions start in the last present '
                    'position: if the first nucleotide\n'
          

In [16]:
pprint(schema_from_file)

{'fields': [{'doc': '* The variant ID.',
             'name': 'id',
             'type': ['null',
                      {'avro.java.string': 'String', 'type': 'string'}]},
            {'default': [],
             'doc': '* Other names used for this genomic variation.',
             'name': 'names',
             'type': {'items': {'avro.java.string': 'String', 'type': 'string'},
                      'type': 'array'}},
            {'doc': '* Chromosome where the genomic variation occurred.',
             'name': 'chromosome',
             'type': {'avro.java.string': 'String', 'type': 'string'}},
            {'doc': '* Normalized position where the genomic variation '
                    'starts.\n'
                    '         * <ul>\n'
                    '         * <li>SNVs have the same start and end '
                    'position</li>\n'
                    '         * <li>Insertions start in the last present '
                    'position: if the first nucleotide\n'
          

In [17]:
pprint(genomic_var_1k[0])

{'alternate': 'T',
 'annotation': {'additionalAttributes': None,
                'alternate': 'T',
                'ancestralAllele': None,
                'chromosome': '22',
                'consequenceTypes': [{'biotype': 'lincRNA',
                                      'cdnaPosition': 0,
                                      'cdsPosition': 0,
                                      'codon': '',
                                      'ensemblGeneId': 'ENSG00000233866',
                                      'ensemblTranscriptId': 'ENST00000424770',
                                      'geneName': 'LA16c-4G1.3',
                                      'proteinVariantAnnotation': {'alternate': '',
                                                                   'features': [],
                                                                   'functionalDescription': None,
                                                                   'keywords': [],
                                 

In [18]:
for f in schema_from_file["fields"]:
    print(f["name"])

id
names
chromosome
start
end
reference
alternate
strand
sv
length
type
hgvs
studies
annotation


## Apache Parquet, ORC and Arrow

We can easily read (decode) and write (encode) data from and to Parquet, ORC and Arrow files interchangeably. The `pyarrow` library allows us to read a Parquet or ORC file into a `pyarrow.Table` object, which is a columnar data structure that can be converted to a Pandas DataFrame. We can also write a `pyarrow.Table` to a Parquet or ORC file.

Parquet has the following types:

- boolean: 1 bit boolean
- int32: 32 bit signed ints
- int64: 64 bit signed ints
- int96: 96 bit signed ints
- float: IEEE 32-bit floating point values
- double: IEEE 64-bit floating point values
- byte_array: arbitrarily long byte arrays
- fixed_len_byte_array: fixed length byte arrays
- string: UTF-8 encoded strings
- enum: enumeration of strings
- temporal: a logical date type

ORC has the following types:

- boolean: 1 bit boolean
- tinyint: 8 bit signed ints
- smallint: 16 bit signed ints
- int: 32 bit signed ints
- bigint: 64 bit signed ints
- float: IEEE 32-bit floating point values
- double: IEEE 64-bit floating point values
- string: UTF-8 encoded strings
- char: ASCII strings
- varchar: UTF-8 strings
- binary: byte arrays
- timestamp: a logical date type
- date: a logical date type
- decimal: arbitrary precision decimals
- list: an ordered collection of objects
- map: a collection of key-value pairs
- struct: an ordered collection of named fields
- union: a list of types

### Reading (Decoding) and Writing (Encoding) a Parquet File

Let's look at how to decode and encode a Parquet file with mock customers data.

In [19]:
import pyarrow as pa
import pyarrow.parquet as pq

In [20]:
table = pq.read_table('../data/userdata1.parquet')

In [21]:
table

pyarrow.Table
registration_dttm: timestamp[ns]
id: int32
first_name: string
last_name: string
email: string
gender: string
ip_address: string
cc: string
country: string
birthdate: string
salary: double
title: string
comments: string
----
registration_dttm: [[2016-02-03 07:55:29.000000000,2016-02-03 17:04:03.000000000,2016-02-03 01:09:31.000000000,2016-02-03 00:36:21.000000000,2016-02-03 05:05:31.000000000,...,2016-02-03 10:30:59.000000000,2016-02-03 17:16:53.000000000,2016-02-03 05:02:20.000000000,2016-02-03 02:41:32.000000000,2016-02-03 09:52:18.000000000]]
id: [[1,2,3,4,5,...,996,997,998,999,1000]]
first_name: [["Amanda","Albert","Evelyn","Denise","Carlos",...,"Dennis","Gloria","Nancy","Annie","Julie"]]
last_name: [["Jordan","Freeman","Morgan","Riley","Burns",...,"Harris","Hamilton","Morris","Daniels","Meyer"]]
email: [["ajordan0@com.com","afreeman1@is.gd","emorgan2@altervista.org","driley3@gmpg.org","cburns4@miitbeian.gov.cn",...,"dharrisrn@eepurl.com","ghamiltonro@rambler.ru","nmor

In [22]:
table.schema

registration_dttm: timestamp[ns]
id: int32
first_name: string
last_name: string
email: string
gender: string
ip_address: string
cc: string
country: string
birthdate: string
salary: double
title: string
comments: string

In [23]:
metadata = pq.read_metadata('../data/userdata1.parquet')

metadata

  created_by: parquet-mr version 1.8.1 (build 4aba4dae7bb0d4edbcf7923ae1339f28fd3f7fcf)
  num_columns: 13
  num_rows: 1000
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 1125

In [24]:
metadata.schema

required group field_id=-1 hive_schema {
  optional int96 field_id=-1 registration_dttm;
  optional int32 field_id=-1 id;
  optional binary field_id=-1 first_name (String);
  optional binary field_id=-1 last_name (String);
  optional binary field_id=-1 email (String);
  optional binary field_id=-1 gender (String);
  optional binary field_id=-1 ip_address (String);
  optional binary field_id=-1 cc (String);
  optional binary field_id=-1 country (String);
  optional binary field_id=-1 birthdate (String);
  optional double field_id=-1 salary;
  optional binary field_id=-1 title (String);
  optional binary field_id=-1 comments (String);
}

In [25]:
metadata.row_group(0).column(10)

  file_offset: 95403
  file_path: 
  physical_type: DOUBLE
  num_values: 1000
  path_in_schema: salary
  is_stats_set: True
  statistics:
      has_min_max: True
      min: 12380.49
      max: 286592.99
      null_count: 68
      distinct_count: 0
      num_values: 932
      physical_type: DOUBLE
      logical_type: None
      converted_type (legacy): NONE
  compression: UNCOMPRESSED
  encodings: ('PLAIN', 'BIT_PACKED', 'RLE')
  has_dictionary_page: False
  dictionary_page_offset: None
  data_page_offset: 95403
  total_compressed_size: 7631
  total_uncompressed_size: 7631

Select the first 3 rows of the table:

In [26]:
table.take([0,1,2])

pyarrow.Table
registration_dttm: timestamp[ns]
id: int32
first_name: string
last_name: string
email: string
gender: string
ip_address: string
cc: string
country: string
birthdate: string
salary: double
title: string
comments: string
----
registration_dttm: [[2016-02-03 07:55:29.000000000,2016-02-03 17:04:03.000000000,2016-02-03 01:09:31.000000000]]
id: [[1,2,3]]
first_name: [["Amanda","Albert","Evelyn"]]
last_name: [["Jordan","Freeman","Morgan"]]
email: [["ajordan0@com.com","afreeman1@is.gd","emorgan2@altervista.org"]]
gender: [["Female","Male","Female"]]
ip_address: [["1.197.201.2","218.111.175.34","7.161.136.94"]]
cc: [["6759521864920116","","6767119071901597"]]
country: [["Indonesia","Canada","Russia"]]
birthdate: [["3/8/1971","1/16/1968","2/1/1960"]]
...

Convert a Table to a DataFrame:

In [27]:
df = table.to_pandas()

In [28]:
df

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 07:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 17:04:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2,2016-02-03 01:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-03 00:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,
4,2016-02-03 05:05:31,5,Carlos,Burns,cburns4@miitbeian.gov.cn,,169.113.235.40,5602256255204850,South Africa,,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2016-02-03 10:30:59,996,Dennis,Harris,dharrisrn@eepurl.com,Male,178.180.111.236,374288806662929,Greece,7/8/1965,263399.54,Editor,
996,2016-02-03 17:16:53,997,Gloria,Hamilton,ghamiltonro@rambler.ru,Female,71.50.39.137,,China,4/22/1975,83183.54,VP Product Management,
997,2016-02-03 05:02:20,998,Nancy,Morris,nmorrisrp@ask.com,,6.188.121.221,3553564071014997,Sweden,5/1/1979,NaN,Junior Executive,
998,2016-02-03 02:41:32,999,Annie,Daniels,adanielsrq@squidoo.com,Female,97.221.132.35,30424803513734,China,10/9/1991,18433.85,Editor,​


You can convert the DataFrame back to a Table (note we're using the method from `pa` which is pyarrow):

In [29]:
new_table = pa.Table.from_pandas(df)

new_table

pyarrow.Table
registration_dttm: timestamp[ns]
id: int32
first_name: string
last_name: string
email: string
gender: string
ip_address: string
cc: string
country: string
birthdate: string
salary: double
title: string
comments: string
----
registration_dttm: [[2016-02-03 07:55:29.000000000,2016-02-03 17:04:03.000000000,2016-02-03 01:09:31.000000000,2016-02-03 00:36:21.000000000,2016-02-03 05:05:31.000000000,...,2016-02-03 10:30:59.000000000,2016-02-03 17:16:53.000000000,2016-02-03 05:02:20.000000000,2016-02-03 02:41:32.000000000,2016-02-03 09:52:18.000000000]]
id: [[1,2,3,4,5,...,996,997,998,999,1000]]
first_name: [["Amanda","Albert","Evelyn","Denise","Carlos",...,"Dennis","Gloria","Nancy","Annie","Julie"]]
last_name: [["Jordan","Freeman","Morgan","Riley","Burns",...,"Harris","Hamilton","Morris","Daniels","Meyer"]]
email: [["ajordan0@com.com","afreeman1@is.gd","emorgan2@altervista.org","driley3@gmpg.org","cburns4@miitbeian.gov.cn",...,"dharrisrn@eepurl.com","ghamiltonro@rambler.ru","nmor

You can write the table back to a Parquet file:

In [30]:
pq.write_table(new_table, "../data/userdata2.parquet")

> 1. How many males and females are there?
>
> 2. What is the average salary for customers from China?
>
> 3. Create a new column `full_name` which combines `first_name` and `last_name` with a space in between in the dataframe. Then convert it back to a new Table and write it to a Parquet file.

### Reading (Decoding) and Writing (Encoding) an ORC File

Let's look at how to decode and encode an ORC file with mock customers data.

In [31]:
import pyarrow as pa
from pyarrow import orc

In [32]:
table2 = orc.read_table('../data/userdata1.orc')

In [33]:
table2

pyarrow.Table
_col0: timestamp[ns]
_col1: int32
_col2: string
_col3: string
_col4: string
_col5: string
_col6: string
_col7: string
_col8: string
_col9: string
_col10: double
_col11: string
_col12: string
----
_col0: [[2016-02-03 13:36:39.000000000,2016-02-03 00:22:28.000000000,2016-02-03 18:29:04.000000000,2016-02-03 13:42:19.000000000,2016-02-03 00:15:29.000000000,...,2016-02-03 13:36:49.000000000,2016-02-03 04:39:01.000000000,2016-02-03 00:33:54.000000000,2016-02-03 00:15:08.000000000,2016-02-03 00:53:53.000000000]]
_col1: [[1,2,3,4,5,...,996,997,998,999,1000]]
_col2: [["Donald","Walter","Michelle","Lori","Howard",...,"Carol","Helen","Stephanie","Marie","Alice"]]
_col3: [["Lewis","Collins","Henderson","Hudson","Miller",...,"Warren","Fields","Sims","Medina","Peterson"]]
_col4: [["dlewis0@clickbank.net","wcollins1@bloglovin.com","mhenderson2@geocities.jp","lhudson3@dion.ne.jp","hmiller4@fema.gov",...,"cwarrenrn@geocities.jp","hfieldsro@comcast.net","ssimsrp@newyorker.com","mmedinarq@t

The column names are missing in the ORC file, so we need to specify them manually, we can use the column names from the previous Table.

In [34]:
table2 = table2.rename_columns(table.column_names)

In [35]:
table2

pyarrow.Table
registration_dttm: timestamp[ns]
id: int32
first_name: string
last_name: string
email: string
gender: string
ip_address: string
cc: string
country: string
birthdate: string
salary: double
title: string
comments: string
----
registration_dttm: [[2016-02-03 13:36:39.000000000,2016-02-03 00:22:28.000000000,2016-02-03 18:29:04.000000000,2016-02-03 13:42:19.000000000,2016-02-03 00:15:29.000000000,...,2016-02-03 13:36:49.000000000,2016-02-03 04:39:01.000000000,2016-02-03 00:33:54.000000000,2016-02-03 00:15:08.000000000,2016-02-03 00:53:53.000000000]]
id: [[1,2,3,4,5,...,996,997,998,999,1000]]
first_name: [["Donald","Walter","Michelle","Lori","Howard",...,"Carol","Helen","Stephanie","Marie","Alice"]]
last_name: [["Lewis","Collins","Henderson","Hudson","Miller",...,"Warren","Fields","Sims","Medina","Peterson"]]
email: [["dlewis0@clickbank.net","wcollins1@bloglovin.com","mhenderson2@geocities.jp","lhudson3@dion.ne.jp","hmiller4@fema.gov",...,"cwarrenrn@geocities.jp","hfieldsro@com

In [36]:
df2 = table2.to_pandas()

df2

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 13:36:39,1.0,Donald,Lewis,dlewis0@clickbank.net,Male,102.22.124.20,,Indonesia,7/9/1972,140249.37,Senior Financial Analyst,
1,2016-02-03 00:22:28,2.0,Walter,Collins,wcollins1@bloglovin.com,Male,247.28.26.93,3587726269478025,China,,NaN,,
2,2016-02-03 18:29:04,3.0,Michelle,Henderson,mhenderson2@geocities.jp,Female,193.68.146.150,,France,1/15/1964,236219.26,Teacher,
3,2016-02-03 13:42:19,4.0,Lori,Hudson,lhudson3@dion.ne.jp,,34.252.168.48,3568840151595649,Russia,4/22/1988,NaN,Nuclear Power Engineer,
4,2016-02-03 00:15:29,5.0,Howard,Miller,hmiller4@fema.gov,Male,103.193.150.230,3583473261055014,France,11/26/1998,50210.02,Senior Editor,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2016-02-03 13:36:49,996.0,Carol,Warren,cwarrenrn@geocities.jp,Female,71.7.191.213,,China,,185421.82,,""""
996,2016-02-03 04:39:01,997.0,Helen,Fields,hfieldsro@comcast.net,Female,164.190.97.183,,Malaysia,,279671.68,,
997,2016-02-03 00:33:54,998.0,Stephanie,Sims,ssimsrp@newyorker.com,Female,135.66.68.181,3548125808139842,Poland,,112275.78,,
998,2016-02-03 00:15:08,999.0,Marie,Medina,mmedinarq@thetimes.co.uk,Female,223.83.175.211,,Kazakhstan,3/25/1969,53564.76,Speech Pathologist,


You can write the table back to an ORC file:

In [37]:
orc.write_table(table2, "../data/userdata2.orc")

> 1. How many males and females are there from China?
>
> 2. Create a new column `age` which is computed from the birthdate in the dataframe. Then convert it back to a new Table and write it to an ORC file.